In [ ]:
import numpy as np
from matplotlib import pyplot as plt
#If you have not installed seaborn uncomment this line
from mpl_toolkits.mplot3d import Axes3D, proj3d
from matplotlib.patches import FancyArrowPatch, Patch
from IPython.display import HTML
from matplotlib import animation
import copy
import time

In [6]:
#functions

# Difference operator (Delta^h operator in discreatized cosserat rod equations)
def modified_diff(t_x):
  """ Modified trapezoidal integration"""
  # Pads a 0 at the end of an array
  temp = np.pad(t_x, (0,1), 
                'constant', 
                constant_values=(0,0)) # Using roll calculate the diff (ghost node of 0)
  return (temp - np.roll(temp, 1))[:-1, :]

# Modified trapezoidal integration (A^h operator in discreatized cosserat rod equations)
def modified_trapz(t_x):
  """ Modified trapezoidal integration"""
  # Pads a 0 at the start of an array
  temp = np.pad(t_x, (1,0), 'constant', constant_values=(0,0))
  # Using roll calculate the integral (ghost node of 0)
  return (0.5*(temp + np.roll(temp, -1)))[1:, :]

# Rodrigues formula, calculates e^(theta*u)
def rodrigues(theta_u):
  """Rodrigues formula, takes input of a 3-d vector, returns rotation matrix (exp(theta*u))"""
  x = np.shape(theta_u)[0]
  exp_theta_u = np.zeros((x,3,3))
  for index in range(x):  
    theta = np.linalg.norm(theta_u[index])
    u = theta_u[index]/theta
    U_mat = np.array([0, -u[2], u[1]],
                    [u[2], 0, -u[0]],
                    [-u[1], u[0], 0])
    exp_theta_u[index] = 1 + np.sin(theta)*U_mat + (1 - np.cos(theta)) * U_mat @ U_mat
  return exp_theta_u

def inv_rodrigues(R):
  theta = np.arccos((np.trace(R)-1)/2)
  K = (R- R.T)/(2*np.sin(theta))
  K_vec = np.array([K[1,2],K[2,0], K[0,1]])
  return K_vec*theta

def inv_rodrigues_mult(R):
  x = np.shape(R)[0]
  theta_u = np.zeros(x,3)
  for index in range(x):
    R_i = R[index]
    theta = np.arccos((np.trace(R_i)-1)/2)
    K = (R_i - R_i.T)/(2*np.sin(theta))
    K_vec = np.array([K[1,2],K[2,0], K[0,1]])
    theta_u[index] = K_vec*theta
  return theta_u

In [5]:
#Equations for calculating various parameters needed for solving cosserat rods
#calculate the edge vectors
def calc_edge_vectors(vertices):
    '''Given the vertices (r), return the edge vectors l_i'''
    edge_vectors = vertices - np.roll(vertices,1, axis = 0)
    return edge_vectors[1:,:]
#calculate the edge length given the edge vectors
def calc_edge_lengths(edge_vectors):
    '''Given the edge lengths (l_i), return the magnitude (|l_i|)'''
    return np.linalg.norm(edge_vectors, axis = 1)

#calculate the tangent vectors for given edge vectors and edge lengths
def calc_tangent_vectors(edge_vectors, edge_lengths):
    '''Given edge vectors (l_i) and edge length (|l_i|), return the tangent vectors (t)'''
    return (edge_vectors.T/edge_lengths).T

def calc_dilatation_factors(edge_lengths, init_edge_lengths):
    '''Given the edge lengths (|l_i|) and initial edge lengths (|l_i^|), return the dilatation factor (e)'''
    return edge_lengths/init_edge_lengths

def getd3(Q):
    return Q[:,2]

def calc_voronoi(edge_lengths):
    voronoi = (edge_lengths + np.roll(edge_lengths, 1))[1:]/2
    return voronoi

#this one is prob weird and needs a recheck
def calc_voronoi_bending(B_hat, voronoi):
    return B_hat * voronoi[0]

def calc_voronoi_dilation(voronoi, voronoi_hat):
    return voronoi/voronoi_hat

def calc_kappa(Q, voronoi):
    x = np.shape(Q)[0]
    kappa = np.zeros((x-1,3))
    for index in range(0,x-1):
        Q_i = Q[index]
        Q_i1 = Q[index+1]
        D_i = voronoi[index]
        kappa[index] = inv_rodrigues(Q_i1@Q_i.T)/D_i
    return kappa

In [ ]:
#initialize all constants needed for calculation!

rho = ... #rod density
E = ...#Young's modulus
G = ...#Shear modulus
F = ...#Force
L = ...#length
r = ...#radius
gamma = ...#dissipation constant
T = ... #simulation time
n = ... #number of discretization points
dt = ... #timestep

nu = (E / G) / 2 - 1 #Poisson's ratio, G = E/2(1+nu)
alpha_c = 4/3 #for a round cross section?
A = np.pi*r**2 #Area

#moments of inertia:
I_1 = np.pi*r**4/4
I_2 = I_1
I_3 = 2*I_1

S_hat = np.array([[alpha_c*G*A, 0, 0],
              [0, alpha_c*G*A, 0],
              [0, 0, E*A]]) #Shearing stiffness matrix
B_hat = np.array([[E*I_1, 0, 0],
              [0, E*I_2, 0],
              [0, 0, G*I_3]])#Bending stiffness matrix
I_hat = np.array([[I_1, 0, 0],
                  [0, I_2, 0],
                  [0, 0, I_3]])

mass_segment = rho * A * L/(n-1)
dm = np.ones(n)*mass_segment
dm[0] = mass_segment/2
dm[-1] = mass_segment/2

J_segment = rho*I_hat*L/(n-1)
dJ_hat = np.ones((n, 3, 3))*J_segment
dJ_hat[0] = J_segment/2
dJ_hat[-1] = J_segment/2

In [ ]:
#set up the vectors/reference frames which will be changing over time
r = np.array([])#node position vector, fill, initial values (there will be n)
v = np.array([])#node linear velocity, fill, initial values (there will be n)
Q = np.array([])#segment local reference frame, fill, initial values (there will be n-1)
omega = np.array([])#segment angular velocity, fill, initial values (there will be n-1)
sigma = np.array([[],[],[]])
dv_dt = np.zeros(n,3)#check shapes, should it be n-1?
dw_dt = np.zeros(n,3)#check shapes, should it be n-1?
kappa_hat = ... # I should fill this in soon! This is important. Need inverse rodrigues!
l_hat = calc_edge_vectors(r)
l_mag_hat = calc_edge_lengths(l_hat)
D_hat = calc_voronoi(l_mag_hat)
B_it_hat = calc_voronoi_bending(D_hat, B_hat)

In [ ]:
#Note: time stepping equation signs should follow the project document info
#update r (half step):
r = r + dt/2 * v
#update Q (half step):
Q = rodrigues(-dt/2*omega) @ Q #need to check if this works properly shape(n-1,3,3)@shape(n-1,3,3)

#find li, li_mag, dilatation factors, etc.
l = calc_edge_vectors(r)
l_mag = calc_edge_lengths(l)
t = calc_tangent_vectors(l, l_mag)
e = calc_dilatation_factors(l, l_hat)
d3 = getd3(Q)
D = calc_voronoi(l_mag)
epsilon = calc_voronoi_dilation(D, D_hat)

#find dv/dt and dw/dt
for index in range(0,n-1): #is this the right number of loops?
    Q_i = Q[index]
    e_i = e[index]
    t_i = t[index]
    l_mag_hat_i = l_hat[index]
    sigma_i = e_i*t_i - d3[index]
    sigma[index] = sigma_i
    epsilon_i = epsilon[index]
    kappa_i = kappa_hat[index]
    voronoi_hat_i = D_hat[index]
    dJ_hat_i = dJ_hat[index]

    #remember these are the discretized equations
    temp_diff = Q_i@S_hat@sigma_i/e_i
    dv_dt[index] = (modified_diff(temp_diff) + F)/dm
    temp_diff = B_it_hat@kappa_hat/epsilon_i**3#check this
    temp_quad = np.cross(kappa_hat, B_it_hat@kappa_hat)*voronoi_hat_i/epsilon_i**3#check
    dw_dt[index] = (modified_diff(temp_diff) + modified_trapz(temp_quad) + np.cross((Q_i@t_i), S_hat@sigma_i)*l_mag_hat_i)/dJ_hat_i

#full step for v, omega
v = v + dt* dv_dt
omega = omega + dt*dw_dt
#full step for r, Q
r = r + dt/2*v
Q = rodrigues(-dt/2*omega)@Q

In [ ]:
r = np.array([[0, 0, 1], [0,0,3], [5,0,3], [5,4,3], [5,6,3], [4,6,2], [3,6,9],[0, 0, 1], [0,0,3], [5,0,3], [5,4,3], [5,6,3], [4,6,2], [3,6,9],[0, 0, 1], [0,0,3], [5,0,3], [5,4,3], [5,6,3], [4,6,2], [3,6,9],[0, 0, 1], [0,0,3], [5,0,3], [5,4,3], [5,6,3], [4,6,2], [3,6,9],[0, 0, 1], [0,0,3], [5,0,3], [5,4,3], [5,6,3], [4,6,2], [3,6,9],[0, 0, 1], [0,0,3], [5,0,3], [5,4,3], [5,6,3], [4,6,2], [3,6,9]])
li = calc_edge_vectors(r)
np.shape(r)
#li_mag = calc_edge_lengths(li)
#t = calc_tangent_vectors(li, li_mag)
#li_mag_new = ([4,10,2,6])

# r = np.array([[0, 0, 1], [0,0,3], [5,0,3], [5,4,3], [5,6,3], [4,6,2], [3,6,9],[0, 0, 1], [0,0,3], [5,0,3], [5,4,3], [5,6,3], [4,6,2], [3,6,9],[0, 0, 1], [0,0,3], [5,0,3], [5,4,3], [5,6,3], [4,6,2], [3,6,9],[0, 0, 1], [0,0,3], [5,0,3], [5,4,3], [5,6,3], [4,6,2], [3,6,9],[0, 0, 1], [0,0,3], [5,0,3], [5,4,3], [5,6,3], [4,6,2], [3,6,9],[0, 0, 1], [0,0,3], [5,0,3], [5,4,3], [5,6,3], [4,6,2], [3,6,9]])
# li = calc_edge_vectors(r)
# # li2, t2 = calc_edge_vectors2(r)
# # print(li,"\n", li2)
# # print(t1, t2)

# edge_lengths = np.array([1,2,4,1,5,6,7])
# calc_voronoi(edge_lengths)
# Q = np.array([[[1,0,0], [0,1,0], [0,0,1]],[[0,1,0], [1,0,0], [0,0,1]], [[0,1,0],[0,0,1],[1,0,0]]])
# Q2 = np.array([[[2,0,0], [0,1,3], [0,0,2]],[[0,3,0], [3,1,0], [0,0,3]], [[0,2,0],[0,0,1],[1.5,0,0]]])
# r = np.array([[0, 1, 0], [0,0,3], [5,0,3], [5,4,3]])
# li = calc_edge_vectors(r)
# li_mag = calc_edge_lengths(li)
# t = calc_tangent_vectors(li, li_mag)
# e = np.array([1,0.5,3])
# d3_0 = getd3(Q)[0]

